# <font color='CC3D3D'> 0.5_4 Visit_Count_Generation
    
- log_data.csv와 user_spec.csv부터 개별 유저가 하루 동안 앱에 몇 번 방문했는지를 집계
- <span style="color:blue"> **visit.csv** </span> 생성

# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
from IPython.display import display
import gc
import warnings
import time

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
def view_info(df:pd.DataFrame) -> None:
    print(df.shape)
    print()
    df.info()
    print()
    display(df.head())

# Data Load

In [ ]:
loan = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/loan_result.csv", engine='c',
                   low_memory=True,
                   parse_dates=['loanapply_insert_time'],
                   dtype={'application_id': 'uint32', 'bank_id': 'uint8', 'product_id': 'uint8'},
                   usecols=['application_id', 'loanapply_insert_time', 'is_applied'])
log = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/log_data.csv", engine='c', 
                  dtype={'user_id': 'uint32', 'event': 'object'},
                  usecols=['user_id', 'event', 'timestamp', 'date_cd'], 
                  low_memory=True,
                  parse_dates=['timestamp', 'date_cd'])
user = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/user_spec.csv", engine='c',
                   dtype={'application_id': 'uint32', 'user_id': 'uint32'},
                   low_memory=True,
                   parse_dates=['insert_time'],
                   usecols=['application_id', 'user_id', 'insert_time'])

view_info(loan)
print()
view_info(log)
print()
view_info(user)

(13527363, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527363 entries, 0 to 13527362
Data columns (total 3 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
dtypes: datetime64[ns](1), float64(1), uint32(1)
memory usage: 258.0 MB



,application_id,loanapply_insert_time,is_applied
0,1748340,2022-06-07 13:05:41,NaN
1,1748340,2022-06-07 13:05:41,NaN
2,1748340,2022-06-07 13:05:41,NaN
3,1748340,2022-06-07 13:05:41,NaN
4,1748340,2022-06-07 13:05:41,NaN



(17843993, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17843993 entries, 0 to 17843992
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    uint32        
 1   event      object        
 2   timestamp  datetime64[ns]
 3   date_cd    datetime64[ns]
dtypes: datetime64[ns](2), object(1), uint32(1)
memory usage: 476.5+ MB



,user_id,event,timestamp,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,2022-03-25



(1394216, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394216 entries, 0 to 1394215
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   application_id  1394216 non-null  uint32        
 1   user_id         1394216 non-null  uint32        
 2   insert_time     1394216 non-null  datetime64[ns]
dtypes: datetime64[ns](1), uint32(2)
memory usage: 21.3 MB



,application_id,user_id,insert_time
0,1249046,118218,2022-06-07 06:28:18
1,954900,553686,2022-06-07 14:29:03
2,137274,59516,2022-06-07 21:40:22
3,1570936,167320,2022-06-07 09:40:27
4,967833,33400,2022-06-07 08:55:07


In [ ]:
app_user = {user['application_id'].iloc[i]: user['user_id'].iloc[i] for i in range(user.shape[0])}
loan['user_id'] = loan['application_id'].map(app_user)
loan.dropna(subset=['user_id'], inplace=True)
loan['user_id'] = loan['user_id'].astype(np.uint32)
loan['date_cd'] = pd.to_datetime(loan['loanapply_insert_time'].dt.date)
loan.sort_values(by=['user_id', 'loanapply_insert_time'], inplace=True)
loan.reset_index(drop=True, inplace=True)
view_info(loan)

(13527250, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527250 entries, 0 to 13527249
Data columns (total 5 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   user_id                uint32        
 4   date_cd                datetime64[ns]
dtypes: datetime64[ns](2), float64(1), uint32(2)
memory usage: 412.8 MB



,application_id,loanapply_insert_time,is_applied,user_id,date_cd
0,248576,2022-05-21 23:41:09,0.0,9,2022-05-21
1,248576,2022-05-21 23:41:09,0.0,9,2022-05-21
2,248576,2022-05-21 23:41:09,0.0,9,2022-05-21
3,248576,2022-05-21 23:41:09,0.0,9,2022-05-21
4,248576,2022-05-21 23:41:10,0.0,9,2022-05-21


In [ ]:
del user
gc.collect()

21

In [ ]:
## apply 2번 이상한 사람들
total_apply = loan[['user_id', 'application_id', 'date_cd']].drop_duplicates()
total_apply = total_apply.groupby(by=['user_id', 'date_cd']).count()

multi_apply = total_apply[total_apply['application_id'] > 1]
multi_apply.sort_values(by=['user_id', 'date_cd'], inplace=True)
multi_apply.reset_index(drop=False, inplace=True)
view_info(multi_apply)

single_apply = total_apply[total_apply['application_id'] == 1]
single_apply.sort_values(by=['user_id', 'date_cd'], inplace=True)
single_apply.reset_index(drop=False, inplace=True)
view_info(single_apply)

(62443, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62443 entries, 0 to 62442
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_id         62443 non-null  uint64        
 1   date_cd         62443 non-null  datetime64[ns]
 2   application_id  62443 non-null  int64         
dtypes: datetime64[ns](1), int64(1), uint64(1)
memory usage: 1.4 MB



,user_id,date_cd,application_id
0,33,2022-03-29,2
1,51,2022-04-29,2
2,65,2022-03-21,2
3,70,2022-06-23,2
4,98,2022-06-16,2


(826089, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826089 entries, 0 to 826088
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         826089 non-null  uint64        
 1   date_cd         826089 non-null  datetime64[ns]
 2   application_id  826089 non-null  int64         
dtypes: datetime64[ns](1), int64(1), uint64(1)
memory usage: 18.9 MB



,user_id,date_cd,application_id
0,9,2022-05-21,1
1,11,2022-03-24,1
2,11,2022-04-20,1
3,11,2022-06-09,1
4,11,2022-06-13,1


# Data Structure

In [ ]:
multi_loan = loan[(loan['user_id'].isin(multi_apply['user_id'].unique())) & (loan['date_cd'].isin(multi_apply['date_cd'].unique()))]
multi_loan['hour'] = multi_loan['loanapply_insert_time'].dt.hour

multi_time = multi_loan[['user_id', 'date_cd', 'hour']].drop_duplicates()
display(multi_time)
print()

mt_group = multi_time[['user_id', 'date_cd', 'hour']].groupby(['user_id', 'date_cd']).count()
mt_group.reset_index(drop=False, inplace=True)
mt_group[mt_group['hour'] == 1]

,user_id,date_cd,hour
329,33,2022-03-29,17
808,51,2022-04-29,7
809,51,2022-04-29,16
968,65,2022-03-21,10
971,65,2022-03-21,13
...,...,...,...
13526489,879649,2022-05-24,21
13526507,879649,2022-05-24,22
13526987,879689,2022-04-05,14
13527020,879689,2022-05-18,21


,user_id,date_cd,hour
0,33,2022-03-29,1
3,65,2022-04-07,1
4,65,2022-04-11,1
5,70,2022-05-24,1
7,98,2022-06-16,1
...,...,...,...
255551,879638,2022-06-21,1
255552,879638,2022-06-28,1
255554,879689,2022-04-05,1
255555,879689,2022-05-18,1


In [ ]:
only_one = mt_group[mt_group['hour'] == 1]
for i, (user, date_cd)in enumerate(zip(only_one['user_id'], only_one['date_cd'])):
    display(loan[(loan['user_id'] == user) & (loan['date_cd'] == date_cd)])
    print()
    if i == 4:
        break

,application_id,loanapply_insert_time,is_applied,user_id,date_cd
329,553073,2022-03-29 17:58:09,0.0,33,2022-03-29
330,553073,2022-03-29 17:58:09,0.0,33,2022-03-29
331,553073,2022-03-29 17:58:09,0.0,33,2022-03-29
332,553073,2022-03-29 17:58:09,0.0,33,2022-03-29
333,553073,2022-03-29 17:58:10,0.0,33,2022-03-29
334,553073,2022-03-29 17:58:10,0.0,33,2022-03-29
335,553073,2022-03-29 17:58:10,0.0,33,2022-03-29
336,553073,2022-03-29 17:58:10,0.0,33,2022-03-29
337,553073,2022-03-29 17:58:10,0.0,33,2022-03-29
338,553073,2022-03-29 17:58:10,0.0,33,2022-03-29


,application_id,loanapply_insert_time,is_applied,user_id,date_cd
973,122628,2022-04-07 17:12:05,0.0,65,2022-04-07
974,122628,2022-04-07 17:12:07,0.0,65,2022-04-07
975,122628,2022-04-07 17:12:07,0.0,65,2022-04-07
976,122628,2022-04-07 17:12:16,0.0,65,2022-04-07


,application_id,loanapply_insert_time,is_applied,user_id,date_cd
977,1748171,2022-04-11 13:14:38,1.0,65,2022-04-11
978,1748171,2022-04-11 13:14:40,0.0,65,2022-04-11
979,1748171,2022-04-11 13:14:51,0.0,65,2022-04-11


,application_id,loanapply_insert_time,is_applied,user_id,date_cd
1251,1546131,2022-05-24 16:51:30,0.0,70,2022-05-24
1252,1546131,2022-05-24 16:51:30,0.0,70,2022-05-24
1253,1546131,2022-05-24 16:51:30,0.0,70,2022-05-24
1254,1546131,2022-05-24 16:51:30,0.0,70,2022-05-24
1255,1546131,2022-05-24 16:51:30,0.0,70,2022-05-24
1256,1546131,2022-05-24 16:51:30,0.0,70,2022-05-24
1257,1546131,2022-05-24 16:51:31,0.0,70,2022-05-24
1258,1546131,2022-05-24 16:51:31,1.0,70,2022-05-24
1259,1546131,2022-05-24 16:51:32,0.0,70,2022-05-24
1260,1546131,2022-05-24 16:51:32,0.0,70,2022-05-24


,application_id,loanapply_insert_time,is_applied,user_id,date_cd
1714,1870874,2022-06-16 02:50:36,NaN,98,2022-06-16
1715,1870874,2022-06-16 02:50:36,NaN,98,2022-06-16
1716,1870874,2022-06-16 02:50:36,NaN,98,2022-06-16
1717,1870874,2022-06-16 02:50:37,NaN,98,2022-06-16
1718,489777,2022-06-16 02:55:22,NaN,98,2022-06-16
1719,489777,2022-06-16 02:55:22,NaN,98,2022-06-16
1720,489777,2022-06-16 02:55:23,NaN,98,2022-06-16
1721,489777,2022-06-16 02:55:24,NaN,98,2022-06-16


In [ ]:
log = log[log['user_id'].isin(loan['user_id'].unique())]
log.sort_values(by=['user_id', 'timestamp'], inplace=True)
log.reset_index(drop=True, inplace=True)
log.info()
print()
log.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14344022 entries, 0 to 14344021
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    uint32        
 1   event      object        
 2   timestamp  datetime64[ns]
 3   date_cd    datetime64[ns]
dtypes: datetime64[ns](2), object(1), uint32(1)
memory usage: 383.0+ MB



,user_id,event,timestamp,date_cd
0,9,GetCreditInfo,2022-05-21 23:37:58,2022-05-21
1,9,GetCreditInfo,2022-05-21 23:43:33,2022-05-21
2,9,GetCreditInfo,2022-05-21 23:43:52,2022-05-21
3,11,OpenApp,2022-03-24 10:53:59,2022-03-24
4,11,GetCreditInfo,2022-03-24 10:54:07,2022-03-24


In [ ]:
loan[loan['user_id'] == 65].head(50)

,application_id,loanapply_insert_time,is_applied,user_id,date_cd
968,840143,2022-03-21 10:38:31,1.0,65,2022-03-21
969,840143,2022-03-21 10:38:32,0.0,65,2022-03-21
970,840143,2022-03-21 10:38:42,0.0,65,2022-03-21
971,1991748,2022-03-21 13:37:16,0.0,65,2022-03-21
972,1991748,2022-03-21 13:37:26,0.0,65,2022-03-21
973,122628,2022-04-07 17:12:05,0.0,65,2022-04-07
974,122628,2022-04-07 17:12:07,0.0,65,2022-04-07
975,122628,2022-04-07 17:12:07,0.0,65,2022-04-07
976,122628,2022-04-07 17:12:16,0.0,65,2022-04-07
977,1748171,2022-04-11 13:14:38,1.0,65,2022-04-11


In [ ]:
log[log['user_id'] == 65]

,user_id,event,timestamp,date_cd
915,65,Login,2022-04-07 17:10:30,2022-04-07
916,65,StartLoanApply,2022-04-07 17:10:36,2022-04-07
917,65,ViewLoanApplyIntro,2022-04-07 17:10:37,2022-04-07
918,65,CompleteIDCertification,2022-04-07 17:10:57,2022-04-07
919,65,EndLoanApply,2022-04-07 17:13:33,2022-04-07
920,65,OpenApp,2022-04-11 13:12:48,2022-04-11
921,65,Login,2022-04-11 13:12:53,2022-04-11
922,65,StartLoanApply,2022-04-11 13:13:11,2022-04-11
923,65,ViewLoanApplyIntro,2022-04-11 13:13:11,2022-04-11
924,65,CompleteIDCertification,2022-04-11 13:13:35,2022-04-11


# Generate

* 로그 데이터가 무지 엉망이다 -> loan apply 시간이랑 맞지도 않고, application별로 구분되지도 않는다
* 따라서 방법은 다음과 같다.
    - 일단 유저의 일간 duration을 구하고, loan 데이터에 붙인다.
    - 결측치 처리 방법은 다음 중 하나
        - train과 test를 다르게 처리한다면, train에 대해서는 apply한 날과 그렇지 않은 날의 user별 duration의 평균을 각각 구해 그 평균으로 채워주고, test에 대해서는 '기대출 수가 많은 사람일수록 신청 확률이 높다'는 가정에 근거해 대출 신청한 application 비율에 따른 가중 평균으로 채워준다.
        - train과 test를 같은 방법으로 처리한다면, 위에 적은 방법 중 후자를 택한다.

In [ ]:
del date_cd, i, mt_group, multi_apply, multi_loan, multi_time, only_one, single_apply, total_apply, user
gc.collect()

105

## 방문 횟수

In [ ]:
user_subdfs = []
for user in tqdm(log['user_id'].unique()):
    subdf = log[log['user_id'] == user]
    subdf.sort_values(by=['user_id', 'timestamp'], inplace=True)
    user_subdfs.append(subdf)

100%|██████████| 293516/293516 [1:12:32<00:00, 67.43it/s]


In [ ]:
threshold = len(user_subdfs) // 5
listA = user_subdfs[:threshold]
listB = user_subdfs[threshold:2*threshold]
listC = user_subdfs[2*threshold:3*threshold]
listD = user_subdfs[3*threshold:4*threshold]
listE = user_subdfs[4*threshold:]

In [ ]:
def visitcnt(array:list) -> pd.DataFrame:
    result_df = pd.DataFrame(columns=['user_id', 'date_cd', 'visit'])
    visit_dt = subdf['date_cd'].unique()
    for item in tqdm(array):
        user = item['user_id'].unique()[0]
        for date_cd in item['date_cd'].unique():
            subsubdf = item[item['date_cd'] == date_cd]
            subsubdf.reset_index(drop=False, inplace=True)
            if subsubdf.shape[0] > 0:
                count = 1
                for i in range(1, subsubdf.shape[0]):
                    timediff = subsubdf['timestamp'].iloc[i] - subsubdf['timestamp'].iloc[i-1]
                    if timediff.seconds > 1800:
                        count += 1
            else:
                count = 0
            result_df = result_df.append({'user_id': np.uint32(user), 'date_cd': pd.to_datetime(date_cd), 'visit': np.uint8(count)}, ignore_index=True)
    return result_df

In [ ]:
resultA = visitcnt(listA)
resultB = visitcnt(listB)
resultC = visitcnt(listC)
resultD = visitcnt(listD)
resultE = visitcnt(listE)

result_df = pd.concat([resultA, resultB, resultC, resultD, resultE])

100%|██████████| 58704/58704 [58:44<00:00, 16.66it/s]


In [ ]:
# result_df = pd.DataFrame(columns=['user_id', 'date_cd', 'visit'])
# users = log['user_id'].unique()
# for item in tqdm():
#     subdf = log[log['user_id'] == user]
#     visit_dt = subdf['date_cd'].unique()
#     for dt in visit_dt:
#         subsubdf = subdf[subdf['date_cd'] == dt]
#         subsubdf.reset_index(drop=False, inplace=True)
#         count = 1
#         for i in range(1, subsubdf.shape[0]):
#             timediff = subsubdf['timestamp'].iloc[i] - subsubdf['timestamp'].iloc[i-1]
#             if timediff.seconds > 1800:
#                 count += 1
#         result_df = result_df.append({'user_id': np.uint32(user), 'date_cd': pd.to_datetime(dt), 'visit': np.uint8(count)}, ignore_index=True)

# Deployment CSV

In [ ]:
result_df.to_csv('../Data/visit.csv', index=False)